In [15]:
import pandas as pd

data = pd.read_csv("/content/btc_dataset.csv")


In [6]:
from sklearn.model_selection import train_test_split


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [10]:
from sklearn.preprocessing import MinMaxScaler


In [11]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()


In [12]:
# Select the features and the target
features = ['open', 'high', 'low', 'Volume BTC', 'Volume USD']
target = 'close'

# Prepare the features and target
X = data[features]
y = data[target]

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features and target
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# We need to reshape y to be a 2D array for the scaler
y_train_reshaped = y_train.values.reshape(-1, 1)
y_test_reshaped = y_test.values.reshape(-1, 1)

y_train_scaled = scaler_y.fit_transform(y_train_reshaped)
y_test_scaled = scaler_y.transform(y_test_reshaped)

X_train_scaled.shape, X_test_scaled.shape, y_train_scaled.shape, y_test_scaled.shape


((35535, 5), (8884, 5), (35535, 1), (8884, 1))

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

# Define the model architecture
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Define the early stopping condition
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
)

# Train the model
history = model.fit(
    X_train_scaled,
    y_train_scaled,
    validation_split=0.2,
    batch_size=32,
    epochs=100,
    callbacks=[early_stopping],
    verbose=1,
)


Epoch 1/100
889/889 [==============================] - 3s 3ms/step - loss: 7.7260e-04 - mae: 0.0064 - val_loss: 5.3422e-06 - val_mae: 0.0013
Epoch 2/100
889/889 [==============================] - 2s 2ms/step - loss: 6.1081e-06 - mae: 0.0014 - val_loss: 6.1492e-06 - val_mae: 0.0015
Epoch 3/100
889/889 [==============================] - 2s 2ms/step - loss: 5.9913e-06 - mae: 0.0014 - val_loss: 5.1726e-06 - val_mae: 0.0013
Epoch 4/100
889/889 [==============================] - 2s 2ms/step - loss: 5.7223e-06 - mae: 0.0014 - val_loss: 5.7434e-06 - val_mae: 0.0014
Epoch 5/100
889/889 [==============================] - 3s 3ms/step - loss: 5.8765e-06 - mae: 0.0015 - val_loss: 4.6396e-06 - val_mae: 0.0011
Epoch 6/100
889/889 [==============================] - 2s 2ms/step - loss: 6.1612e-06 - mae: 0.0016 - val_loss: 4.7118e-06 - val_mae: 0.0012


In [20]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# Get the closing prices
closing_prices = data['close'].values.reshape(-1, 1)

# Scale the closing prices
scaler = MinMaxScaler()
closing_prices = scaler.fit_transform(closing_prices)

# Split the data into 90% for training and 10% for testing
train_size = int(len(closing_prices) * 0.9)
train, test = closing_prices[0:train_size], closing_prices[train_size:len(closing_prices)]

# Convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

# Reshape into X=t and Y=t+1
look_back = 10
X_train, y_train = create_dataset(train, look_back)
X_test, y_test = create_dataset(test, look_back)

# Reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Define the LSTM model
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=2)

# Make predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Invert predictions
train_predict = scaler.inverse_transform(train_predict)
y_train = scaler.inverse_transform([y_train])
test_predict = scaler.inverse_transform(test_predict)
y_test = scaler.inverse_transform([y_test])


Epoch 1/100
39966/39966 - 72s - loss: 4.4875e-04 - 72s/epoch - 2ms/step
Epoch 2/100
39966/39966 - 68s - loss: 5.1224e-05 - 68s/epoch - 2ms/step
Epoch 3/100
39966/39966 - 72s - loss: 3.8790e-05 - 72s/epoch - 2ms/step
Epoch 4/100
39966/39966 - 74s - loss: 3.4216e-05 - 74s/epoch - 2ms/step
Epoch 5/100
39966/39966 - 71s - loss: 3.0864e-05 - 71s/epoch - 2ms/step
Epoch 6/100
39966/39966 - 70s - loss: 2.9187e-05 - 70s/epoch - 2ms/step
Epoch 7/100
39966/39966 - 71s - loss: 2.8980e-05 - 71s/epoch - 2ms/step
Epoch 8/100
39966/39966 - 70s - loss: 2.7589e-05 - 70s/epoch - 2ms/step
Epoch 9/100
39966/39966 - 70s - loss: 2.6653e-05 - 70s/epoch - 2ms/step
Epoch 10/100
39966/39966 - 69s - loss: 2.6089e-05 - 69s/epoch - 2ms/step
Epoch 11/100
39966/39966 - 70s - loss: 2.5811e-05 - 70s/epoch - 2ms/step
Epoch 12/100
39966/39966 - 72s - loss: 2.4834e-05 - 72s/epoch - 2ms/step
Epoch 13/100
39966/39966 - 71s - loss: 2.4823e-05 - 71s/epoch - 2ms/step
Epoch 14/100
39966/39966 - 71s - loss: 2.4136e-05 - 71s/epoc